### Derived Variable 파생변수  
파생변수는 데이터의 특성을 이용하여 분석 효율을 높이는 것이기 때문에 전체 데이터에 대한 파악이 중요할 뿐만 아니라 해당 비즈니스 도메인에 대한 충분한 이해가 수반되어야 한다.  
ex) 온라인 쇼핑몰 판매랑 예측 시 기존 방문 횟수, 클릭 횟수 변수에 전주 대비 방문 횟수 증감률, 전년도 대비 클릭 횟수 증감률 등의 파생변수를 만들어서 모델 예측력을 높일 수 있다.  
파생변수는 기존의 변수를 활용해서 만들어낸 변수이기 때문에 다중공선성 문제가 발생할 가능성이 높다. 그렇기 때문에 파생변수를 만든 다음에는 상관분석을 통해 변수 간의 상관성을 확인해야 한다.

In [2]:
import math
from sklearn import preprocessing
import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# https://www.kaggle.com/datasets/marian447/retail-store-sales-transactions
df = pd.read_csv("../input/retail-store-sales-transactions/scanner_data.csv")

In [4]:
df.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


In [5]:
# 두 개의 변수 결합한 파생변수 생성

# 개당 가격 칼럼 생성
df['Unit_amount'] = df['Sales_Amount']/df['Quantity']

# 총 구매가격 칼럼 생성
df['All_amount'] = df[['Quantity', 'Sales_Amount']].apply(lambda series: series.prod(), axis=1)

In [6]:
df.tail()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount
131701,131702,04/07/2016,20203,32900,IEV,FO112,3.0,6.46,2.153333,19.38
131702,131703,04/07/2016,20203,32900,N8U,I36F2,1.0,4.50,4.500000,4.50
131703,131704,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19,5.190000,5.19
131704,131705,04/07/2016,20203,32900,0H2,ZVTO4,1.0,4.57,4.570000,4.57
131705,131706,04/07/2016,20203,32900,Q4N,QM9BP,1.0,13.68,13.680000,13.68


In [7]:
# 로그, 제곱근, 제곱 변환 파생변수 생성

# 방법1.Sales_Amount 컬럼 로그 적용 (+1)
df['Sales_Amount_log'] = preprocessing.scale(np.log(df['Sales_Amount']+1))
# log 적용 후 scaling

# 방법2.Sales_Amount 컬럼 로그 적용 (+1)
df['Sales_Amount_log2'] = df[['Sales_Amount']].apply(lambda x: np.log(x+1))    

# Sales_Amount 컬럼 제곱근 적용 (+1)
df['Sales_Amount_sqrt'] = np.sqrt(df['Sales_Amount']+1)

# Sales_Amount 컬럼 제곱 적용
df['Sales_Amount_pow'] = pow(df[['Sales_Amount']],2)

df.tail()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,Sales_Amount_log,Sales_Amount_log2,Sales_Amount_sqrt,Sales_Amount_pow
131701,131702,04/07/2016,20203,32900,IEV,FO112,3.0,6.46,2.153333,19.38,-0.204581,2.009555,2.731300,41.7316
131702,131703,04/07/2016,20203,32900,N8U,I36F2,1.0,4.50,4.500000,4.50,-0.592329,1.704748,2.345208,20.2500
131703,131704,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19,5.190000,5.19,-0.441982,1.822935,2.487971,26.9361
131704,131705,04/07/2016,20203,32900,0H2,ZVTO4,1.0,4.57,4.570000,4.57,-0.576240,1.717395,2.360085,20.8849
131705,131706,04/07/2016,20203,32900,Q4N,QM9BP,1.0,13.68,13.680000,13.68,0.656548,2.686486,3.831449,187.1424


In [10]:
# 월 합계, 평균 구매금액 변수 생성

# date 컬럼 날짜형식 변환
df['Date2']= pd.to_datetime(df['Date'], format='mixed') 

# 연도 컬럼 생성
df['Year'] = df['Date2'].dt.year

# 월 컬럼 생성
df['Month'] = df['Date2'].dt.month

#연월별, 고객별 매출 합계, 평균 컬럼 생성
df_sm = df.groupby(['Year', 
                    'Month', 
                    'Customer_ID'])['Sales_Amount'].agg(['sum','mean']).reset_index()

# 기존 일별 테이블에 평균 테이블 조인
df2 = pd.merge(df, df_sm, how='left')

df2.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,Sales_Amount_log,Sales_Amount_log2,Sales_Amount_sqrt,Sales_Amount_pow,Date2,Year,Month,sum,mean
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,3.13,3.13,-0.956750,1.418277,2.032240,9.7969,2016-02-01,2016,2,24.76,6.190000
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,5.46,5.46,-0.387670,1.865629,2.541653,29.8116,2016-02-01,2016,2,5.46,5.460000
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.35,6.35,-0.223478,1.994700,2.711088,40.3225,2016-02-01,2016,2,73.17,5.628462
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,5.59,5.59,-0.362325,1.885553,2.567100,31.2481,2016-02-01,2016,2,171.15,8.150000
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,6.88,6.88,-0.134904,2.064328,2.807134,47.3344,2016-02-01,2016,2,6.88,6.880000


In [12]:
# 월 평균 구매금액 대비 일 별 구매금액 차이 변수 생성
df2['Sales_Amount_Diff'] = df2['mean'] - df2['Sales_Amount']

# 월 평균 구매금액 대비 일 별 구매금액 비율 변수 생성
df2['Sales_Amount_UD'] = df2['Sales_Amount'] / df2['mean']
    
# 월 총 구매금액 대비 일 별 구매금액 비율 변수 생성
df2['Sales_Amount_Rto'] = df2['Sales_Amount']/df2['sum']
    
df2.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,...,Sales_Amount_sqrt,Sales_Amount_pow,Date2,Year,Month,sum,mean,Sales_Amount_Diff,Sales_Amount_UD,Sales_Amount_Rto
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,3.13,3.13,...,2.032240,9.7969,2016-02-01,2016,2,24.76,6.190000,3.060000,0.505654,0.126414
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,5.46,5.46,...,2.541653,29.8116,2016-02-01,2016,2,5.46,5.460000,0.000000,1.000000,1.000000
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.35,6.35,...,2.711088,40.3225,2016-02-01,2016,2,73.17,5.628462,-0.721538,1.128195,0.086784
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,5.59,5.59,...,2.567100,31.2481,2016-02-01,2016,2,171.15,8.150000,2.560000,0.685890,0.032661
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,6.88,6.88,...,2.807134,47.3344,2016-02-01,2016,2,6.88,6.880000,0.000000,1.000000,1.000000


In [14]:
# 전 월 값 파생변수 생성

# 4주 뒤 시점 컬럼 생성
df2['Date2_1_m'] = df2['Date2'] + timedelta(weeks=4)

# # 4주 뒤 시점연도 컬럼 생성
df['Year_1_m'] = df2['Date2_1_m'].dt.year

# # 4주 뒤 시점월 컬럼 생성
df['Month_1_m'] = df2['Date2_1_m'].dt.month

# 4주 전 구매금액 연월별, 고객별 매출 평균 컬럼 생성
df_Mn_1 = df.groupby(['Year_1_m', 
                      'Month_1_m', 
                      'Customer_ID'])['Sales_Amount'].agg(['sum',
                                                           'mean']).reset_index()

# 조인을 위한 컬럼명 변경 
df_Mn_1.rename(columns={'Year_1_m':'Year', 
                        'Month_1_m':'Month', 
                        'sum':"sum_1_m", 
                        'mean':'mean_1_m'}, inplace=True)

df2 = pd.merge(df2, df_Mn_1, how='left')

df2.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,...,Year,Month,sum,mean,Sales_Amount_Diff,Sales_Amount_UD,Sales_Amount_Rto,Date2_1_m,sum_1_m,mean_1_m
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,3.13,3.13,...,2016,2,24.76,6.190000,3.060000,0.505654,0.126414,2016-02-29,14.32,4.773333
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,5.46,5.46,...,2016,2,5.46,5.460000,0.000000,1.000000,1.000000,2016-02-29,14.05,3.512500
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.35,6.35,...,2016,2,73.17,5.628462,-0.721538,1.128195,0.086784,2016-02-29,107.83,7.188667
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,5.59,5.59,...,2016,2,171.15,8.150000,2.560000,0.685890,0.032661,2016-02-29,13.21,6.605000
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,6.88,6.88,...,2016,2,6.88,6.880000,0.000000,1.000000,1.000000,2016-02-29,6.88,6.880000


In [17]:
# 전 월과의 차이 파생변수 생성

# 전 월 대비 구매금액 평균 차이 변수 생성
df2['Mn_diff_1_mean'] = df2['mean'] - df2['mean_1_m']

# 전 월 대비 총 구매금액 차이 변수 생성
df2['Mn_diff_1_sum'] = df2['sum'] - df2['sum_1_m']

df2.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,...,sum,mean,Sales_Amount_Diff,Sales_Amount_UD,Sales_Amount_Rto,Date2_1_m,sum_1_m,mean_1_m,Mn_diff_1_mean,Mn_diff_1_sum
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,3.13,3.13,...,24.76,6.190000,3.060000,0.505654,0.126414,2016-02-29,14.32,4.773333,1.416667,10.44
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,5.46,5.46,...,5.46,5.460000,0.000000,1.000000,1.000000,2016-02-29,14.05,3.512500,1.947500,-8.59
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.35,6.35,...,73.17,5.628462,-0.721538,1.128195,0.086784,2016-02-29,107.83,7.188667,-1.560205,-34.66
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,5.59,5.59,...,171.15,8.150000,2.560000,0.685890,0.032661,2016-02-29,13.21,6.605000,1.545000,157.94
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,6.88,6.88,...,6.88,6.880000,0.000000,1.000000,1.000000,2016-02-29,6.88,6.880000,0.000000,0.00
